In [9]:
from ml_sentiment import *
import eli5
import pandas as pd
filedir = 'data'
train_data, dev_data = get_training_and_dev_data(filedir)
pd.set_option('display.max_colwidth', -1)

def get_error_type(pred, label):
    # return the type of error: tp,fp,tn,fn
    if pred == label:
        return "tp" if pred == '1' else "tn"
    return "fp" if pred == '1' else "fn"

In [10]:
# Change this for your different classifiers
classifier1 = load_classifier('lr_default.pkl')
classifier2 = load_classifier('rf_custom.pkl')

In [11]:
# Create pandas dataframe
predictions = pd.DataFrame.from_dict(dev_data.data)

# Classify data points using classifier1
predictions['Classifier1Prediction'] = classifier1.predict(predictions['Review'])
predictions['Classifier1ErrorType'] = predictions.apply(lambda row: get_error_type(row['Classifier1Prediction'], row['Category']), axis=1)

# Classify data points using classifier 2
predictions['Classifier2Prediction'] = classifier2.predict(predictions['Review'])
predictions['Classifier2ErrorType'] = predictions.apply(lambda row: get_error_type(row['Classifier2Prediction'], row['Category']), axis=1)

In [12]:
# Get metrics for each classifier
def print_metrics(error_type_counts):
    accuracy = (error_type_counts['tp'] + error_type_counts['tn']) / sum(error_type_counts)
    precision = error_type_counts['tp'] / (error_type_counts['tp'] + error_type_counts['fp'])
    recall = error_type_counts['tp'] / (error_type_counts['tp'] + error_type_counts['fn'])
    print("Accuracy:", accuracy, "\nPrecision:", precision, "\nRecall:", recall, "\nF1:", 2 * precision*recall/(precision + recall))

print("Classifier1 Metrics")
print_metrics(predictions['Classifier1ErrorType'].value_counts())
print("\nClassifier2 Metrics")
print_metrics(predictions['Classifier2ErrorType'].value_counts())

Classifier1 Metrics
Accuracy: 0.96068 
Precision: 0.96093812535 
Recall: 0.9604 
F1: 0.960668987316

Classifier2 Metrics
Accuracy: 0.9644 
Precision: 0.962181528662 
Recall: 0.9668 
F1: 0.964485235435


In [13]:
eli5.show_weights(classifier1, top=25)

In [14]:
eli5.show_weights(classifier2, top=25)

Weight,Feature
0.0092 ± 0.0254,bag_of_words:bad
0.0071 ± 0.0148,bag_of_words:worst
0.0058 ± 0.0122,bag_of_words:great
0.0045 ± 0.0142,bag_of_words:terrible
0.0039 ± 0.0043,bag_of_words:and
0.0038 ± 0.0026,custom_feats:length
0.0038 ± 0.0042,custom_feats:pos_count
0.0038 ± 0.0075,bag_of_words:waste
0.0036 ± 0.0067,bag_of_words:no
0.0035 ± 0.0018,custom_feats:num_words


In [15]:
# See some examples of errors for each classifier
# Modify the code to get false negatives and errors for Classifier2)
predictions[predictions['Classifier1ErrorType'] == 'fp'].sample(10)

,Category,FileIndex,Review,Classifier1Prediction,Classifier1ErrorType,Classifier2Prediction,Classifier2ErrorType
23319,0,23319,"Probable reasons why so many people on this site have enjoyed this:<br /><br />1. They might not have read the book. 2. They might enjoy gore and violence in a film. 3. They might be very young and therefore not understand the violence. 4. People might not understand how somehow more scary and more violent it is compared to the original book. 5. There are sure to be many other reasons not covered here.<br /><br />The only thing I liked about this film is the song ""'Bright Eyes"".<br /><br />If perchance, you happen to be one of those people who has read the book, enjoys calm and peaceful films without violence and are quite old and understand scariness and violence, you are sure not to like this. Otherwise you will almost definitely enjoy this.<br /><br />Like in the book, a rabbit called Fiver in an unsuspecting warren warns of terrible danger to come. Only a few rabbits - including his brother Hazel - believe him and they set out on a dangerous journey to find a new place to live...",1,fp,0,tn
21287,0,21287,"The Ladies Man is a funny movie. There's not much thought behind it, but what do you expect from an SNL movie? It's actually better than most SNL movies (i.e. Superstar or A Night At The Roxbury) Tim Meadows and Will Ferrell were both very funny. Chris Parnell was also funny in his short scene (one of the funnier ones in the movie). Other than that, the rest of the cast is average and is just there to support Meadows. I've definitely seen funnier movies, but I've seen dumber ones too. Again, it's not exactly a deep movie, but it's good for a few laughs. It was funnier as a skit though. But still, if you're looking for a pretty funny movie, I'd recommend this one. Just don't think about it too much, or you'll hate it.<br /><br />Rating: 6/10",1,fp,1,fp
4080,0,4080,"Before I start, I _love_ Eddie Izzard. I think he's one of the funniest stand-ups around today. Possibly that means I'm going into this with too high expectations, but I just didn't find Eddie funny in this outing.<br /><br />I think the main problem is Eddie is trying too hard to be Eddie. Everyone knows him as a completely irrelevant comic, and we all love him for it. But in Circle, he appears to be going more for irrelevant than funny, and completely lost me in places. Many of the topics he covers he has covered before - I even think I recognised a few recycled jokes in there.<br /><br />If you buy the DVD you'll find a behind-the-scenes look at Eddie's tour (interesting in places, but not very funny), and a French language version of one of his shows. Die-hards will enjoy seeing Eddie in a different language, but subtitled comedy isn't very funny.<br /><br />If you're a fan of Eddie you've either got this already or you're going to buy it whatever I say. If you're just passing through, buy Glorious or Dressed to Kill - you won't be disappointed. With Circle, you probably will.",1,fp,0,tn
24725,0,24725,"Come on Tina Fey you can do better then this. As soon as the movie started i knew how it would end. Sure it was funny at times. Even laugh out loud funny. But there isn't enough laughs to save this movie. I don't recommend buying this. At the most i recommend renting it but thats all. Baby Mama has some funny scenes but is predictable and fails to have the heartwarming ending it strives for.<br /><br />Tina Fey and Amy Poalher made a good team. Mean Girls is one of my favorite movies. Tina Fey and Amy Poalher both did great in that and they do good in this. But this isn't there best. Baby Mama had a great supporting cast. Dane Cook, Sigourney Weaver and Steve Martin add to the casts greatness.<br /><br />Another pregnancy movie has hit the cinema world. After the great Knocked Up and Juno, Baby Mama looks very average when compared. Knocked Up and Juno are Hilarious, Heartwarming and have endings that leave you with a smile on your 

In [16]:
# See where they disagree
# Modify the code to find cases where one classifier's prediction is correct but the other is incorrect
predictions['ClassifiersAgree'] = predictions['Classifier1Prediction'] == predictions['Classifier2Prediction']
disagreements = predictions[predictions['ClassifiersAgree'] == False]
print("# Cases where the two classifiers disagree:", len(disagreements), "->", len(disagreements) / len(predictions) * 100, "%")
disagreements.sample(10)

# Cases where the two classifiers disagree: 1207 -> 4.828 %


,Category,FileIndex,Review,Classifier1Prediction,Classifier1ErrorType,Classifier2Prediction,Classifier2ErrorType,ClassifiersAgree
23894,0,23894,"Cast to die for in a movie that is considerably less. Vanessa Redgrave is dying but before she goes she begins to tell her daughters the story of her life and of her secret love...<br /><br />This is one of those movies which has the look and expectations of being a great film simply because they have so many great actors and actresses in it so it seems to be about something other than the potboiler that it really is. Not bad as such but with Redgrave, Toni Collette, Glenn Close, Meryl Streep, Clare Danes,Natasha Richardson,Eileen Atkins, Patrick Wilson,Hugh Darcy and others (all giving fine performances) you expect more than a weepy story thats a bit more than a harlequin romance.<br /><br />Wait for Cable.",1,fp,0,tn,False
21819,0,21819,"This movie tries to hard to be something that it's not....a good movie. It wants you to be fooled from begining to end,But fails.From when it starts to get interesting it falls apart and you're just hoping the ending gives you some clue of just what is going on but it didn't.<br /><br />",0,tn,1,fp,False
23657,0,23657,"I wouldn't say this totally sucked, but if it wasn't for Netflix I wouldn't even have this in my house. Steve Martin's eccentric president of a chain of health food stores falls flat. He's just not funny. He's another in a LONG slew of SNL rejects that can only find work whoring themselves to the next SNL movie. The birthing coach with the Elmer Fudd lisp is about as funny as it is original. Amy Poehler simply goes through the same motions she would for a 7 minute SNL skit which is about as funny as SNL lately. The only thing going for this movie is that Tina Fey is easy on the eyes. The ending was predictable as soon as you heard her character couldn't get pregnant. The subject matter could have opened up to other comedic attempts, but it seems to simple simmer along, not really entertaining or creating laughs.",0,tn,1,fp,False
22537,1,22537,"This remarkable film can be summed up very easily. First of all, while the comparisons to ""Princess Bride"" are inevitable, it's almost futile to do so. While both films combine adult wit and humor with a fairy tale backdrop, ""Stardust"" is so much different than any other fantasy/sci-fi film I've ever seen. It's such a hybrid of those genres, but its plot and script are so unique that--along with the performances, special effects, cinematography, and score--the finished product is simply not all that comparable to anything that has ever appeared on the silver screen. Secondly, the score is very effective at simultaneously pulling us into the story and the fantasy world in which it takes place and pushing the story along, while creating just the right amount of awe and excitement necessary to make the magic believable within the realm where the characters exist. Thirdly, I did not find the film to be even remotely difficult to follow or confusing in any way. In fact, the interesting interplay between the three main subplots actually made it even that much more compelling to watch. Wonderfully casted, and superbly acted across the board. This fantasy adventure (with sci-fi elements) was the best one I've seen since ""Return of the King"" (not that I am comparing the two at all). OK, so its not that easy to sum up, but don't let any crude and/or heartless and cynical review nor the film's pathetic PR prevent you from partaking in the best time you could have at the movies this summer (or even possibly in a long time)!",1,tp,0,fn,False
15016,1,15016,"Just a note to add to the above comment. Fear of a Black Hat doesn't have the criminal who's image has been ripped off by the band, that's in CB4. Easily confused as the two films are so similar, but Black Hat is vastly the superior of the two..... yeah.",0,fn,1,tp,False
735,1,735,"Four teenage girlfriends drive to Fort Laurdale for spring break.Unfo